In [1]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, f, f_oneway, chi2_contingency, ttest_1samp

In [2]:
def get_data_from_sheets(excel_file):
    return {sheet_name: excel_file.parse(sheet_name) 
          for sheet_name in excel_file.sheet_names}

In [3]:
index_file_path = '../Data/Index_Data.xlsx'

index_data = pd.ExcelFile(index_file_path)

index_sheets = get_data_from_sheets(index_data)



MF_file_path = '../Data/Mutual_Funds_Data.xlsx'

MF_data = pd.ExcelFile(MF_file_path)

MF_sheets = get_data_from_sheets(MF_data)



GB_file_path = '../Data/Gold_Bond_Data.xlsx'

GB_data = pd.ExcelFile(GB_file_path)

GB_sheets = get_data_from_sheets(GB_data)

## 1. Two Sample T-test

### Null Hypothesis: The mean closing price of `NSEI` is equal to `BSESN`.
### Alternative Hypothesis: The mean closing price of `NSEI` is not equal to `BSESN`.

In [6]:
NSEI = index_sheets['NSEI']
BSESN = index_sheets['BSESN']

In [7]:
print(NSEI['Close'].mean())
print(BSESN['Close'].mean())

10216.909506408463
23484.976475967786


In [8]:
t_stat, test1_p_val = ttest_ind(NSEI['Close'], BSESN['Close'])
print(f"T-Statistic: {t_stat}, p-value: {test1_p_val}")

T-Statistic: -41.72551311100267, p-value: 0.0


In [9]:
def test_hypothesis(p_val):
    if p_val < 0.05:
        return "Reject the null hypothesis."
    else:
        return "Fail to reject the null hypothesis."

In [10]:
test_hypothesis(test1_p_val)

'Reject the null hypothesis.'

#### Here, we have rejected the null hypothesis.
#### The mean closing price of `NSEI` is not equal to `BSESN`.

## 2. F-Test

### Null Hypothesis: The variances of returns of `CNXIT` and `CNXAUTO` are equal.
### Alternative Hypothesis: The variances of returns of `CNXIT` and `CNXAUTO` are not equal.

In [11]:
CNXIT = index_sheets['CNXIT']
CNXAUTO = index_sheets['CNXAUTO']

In [12]:
CNXIT.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock_Splits', 'Percent_Change_In_Price', 'Momentum_Indicator',
       'High_Low_Change', 'High_Open_Change', 'Low_Open_Change', 'Day',
       'Month', 'Year'],
      dtype='object')

In [13]:
CNXIT_variance = np.var(CNXIT['Percent_Change_In_Price'], ddof=1)
CNXAUTO_variance = np.var(CNXAUTO['Percent_Change_In_Price'], ddof=1)

f_value = CNXIT_variance / CNXAUTO_variance

df1 = len(CNXIT['Percent_Change_In_Price']) - 1
df2 = len(CNXAUTO['Percent_Change_In_Price']) - 1

# cumulative distribution function (CDF)
# Survival function = 1 - cdf
test2_p_val = 2 * min(f.cdf(f_value, df1, df2), f.sf(f_value, df1, df2))

print('Degree of freedom 1:',df1)
print('Degree of freedom 2:',df2)
print("F-statistic:", f_value)
print("p-value:", test2_p_val)

Degree of freedom 1: 3982
Degree of freedom 2: 3334
F-statistic: 94.39910971668863
p-value: 0.0


In [14]:
test_hypothesis(test2_p_val)

'Reject the null hypothesis.'

#### Here, reject the null hypothesis.

#### It means that The variances of returns of `CNXIT` and `CNXAUTO` are not equal

## 3. One-way ANOVA

### Null Hypothesis: The mean returns of all mutual funds are the same.
### Alternative Hypothesis: At least one mutual fund has a different mean return.

In [16]:
f_stat, test3_p_val = f_oneway(
    *[df['Percent_Change_In_Price'].dropna() for df in MF_sheets.values()]
)
print(f"F-Statistic: {f_stat}, p-value: {test3_p_val}")

F-Statistic: 0.4446922623860055, p-value: 0.975063014532883


In [17]:
test_hypothesis(test3_p_val)

'Fail to reject the null hypothesis.'

#### Here. we fail to reject the null hypothesis.

#### Because of not having sufficient evidences, we are not able to prove that At least one mutual fund has a different mean return.

## 4. Chi-Square Test

### Null Hypothesis: Gold bond returns are independent of market conditions (e.g., bearish or bullish).
### Alternative Hypothesis: Gold bond returns are not independent of market conditions.

In [18]:
GoldBond = GB_sheets['Gold Bond']

In [19]:
GoldBond['Gold_Return_Category'] = GoldBond['Percent_Change_In_Price'].apply(
    lambda x: 'Positive' if x > 0 else 'Negative'
)

NSEI['Market_Condition'] = NSEI['Percent_Change_In_Price'].apply(
    lambda x: 'Bullish' if x > 0.005 else ('Bearish' if x < -0.005 else 'Neutral')
)

In [20]:
GoldBond.head()

,Date,Series,Open,High,Low,Prev_Close,Ltp,Close,Vwap,52W_H,...,No_Of_Trades,Day,Month,Year,Percent_Change_In_Price,Momentum_Indicator,High_Low_Change,High_Open_Change,Low_Open_Change,Gold_Return_Category
0,2019-12-30,GB,3700.00,3700.00,3700.00,3599.50,3700.00,3700.00,3700.00,3840.0,...,1,30,12,2019,NaN,0.000000,0.000000,0.000000,0.000000,Negative
1,2019-12-23,GB,3750.00,3750.00,3599.00,3589.00,3599.00,3599.50,3616.33,3840.0,...,5,23,12,2019,-0.027162,-0.040133,0.041956,0.000000,-0.040267,Negative
2,2019-12-17,GB,3590.00,3590.00,3589.00,3580.05,3589.00,3589.00,3589.40,3840.0,...,3,17,12,2019,-0.002917,-0.000279,0.000279,0.000000,-0.000279,Negative
3,2019-12-16,GB,3580.05,3580.05,3580.05,3580.00,3580.05,3580.05,3580.05,3840.0,...,1,16,12,2019,-0.002494,0.000000,0.000000,0.000000,0.000000,Negative
4,2019-12-12,GB,3562.00,3580.00,3562.00,3585.00,3580.00,3580.00,3565.00,3840.0,...,2,12,12,2019,-0.000014,0.005053,0.005053,0.005053,0.000000,Negative


In [21]:
GoldBond.tail()

,Date,Series,Open,High,Low,Prev_Close,Ltp,Close,Vwap,52W_H,...,No_Of_Trades,Day,Month,Year,Percent_Change_In_Price,Momentum_Indicator,High_Low_Change,High_Open_Change,Low_Open_Change,Gold_Return_Category
678,2024-01-15,GB,6240.01,6400.0,6240.01,6220.0,6400.0,6400.0,6293.34,6400.0,...,2,15,1,2024,0.024000,0.025639,0.025639,0.025639,0.0,Positive
679,2024-01-11,GB,6220.00,6220.0,6220.00,6275.0,6220.0,6220.0,6220.00,6340.0,...,1,11,1,2024,-0.028125,0.000000,0.000000,0.000000,0.0,Negative
680,2024-01-09,GB,6275.00,6275.0,6275.00,6275.0,6275.0,6275.0,6275.00,6340.0,...,2,9,1,2024,0.008842,0.000000,0.000000,0.000000,0.0,Positive
681,2024-01-08,GB,6275.00,6275.0,6275.00,6340.0,6275.0,6275.0,6275.00,6340.0,...,2,8,1,2024,0.000000,0.000000,0.000000,0.000000,0.0,Negative
682,2024-01-05,GB,6340.00,6340.0,6340.00,6340.0,6340.0,6340.0,6340.00,6340.0,...,2,5,1,2024,0.010359,0.000000,0.000000,0.000000,0.0,Positive


In [22]:
NSEI.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Percent_Change_In_Price,Momentum_Indicator,High_Low_Change,High_Open_Change,Low_Open_Change,Day,Month,Year,Market_Condition
4267,2025-02-11 00:00:00+05:30,23383.550781,23390.050781,22986.650391,23071.800781,268000,0,0,-0.013250,-0.013332,0.017549,0.000278,-0.016973,11,2,2025,Bearish
4268,2025-02-12 00:00:00+05:30,23050.800781,23144.699219,22798.349609,23045.250000,279700,0,0,-0.001151,-0.000241,0.015192,0.004074,-0.010952,12,2,2025,Neutral
4269,2025-02-13 00:00:00+05:30,23055.750000,23235.500000,22992.199219,23031.400391,265700,0,0,-0.000601,-0.001056,0.010582,0.007796,-0.002756,13,2,2025,Neutral
4270,2025-02-14 00:00:00+05:30,23096.449219,23133.699219,22774.849609,22929.250000,254500,0,0,-0.004435,-0.007239,0.015756,0.001613,-0.013924,14,2,2025,Neutral
4271,2025-02-17 00:00:00+05:30,22809.900391,22974.199219,22725.449219,22959.500000,0,0,0,0.001319,0.006559,0.010946,0.007203,-0.003702,17,2,2025,Neutral


In [23]:
NSEI['Date'] = pd.to_datetime(NSEI['Date'])
NSEI['Date'] = NSEI['Date'].dt.tz_localize(None)

In [24]:
merged_data = pd.merge(GoldBond[['Date', 'Gold_Return_Category']],
                       NSEI[['Date', 'Market_Condition']],
                       on='Date')

contingency_table = pd.crosstab(merged_data['Gold_Return_Category'], 
                                merged_data['Market_Condition'])
print(contingency_table)

Market_Condition      Bearish  Bullish  Neutral
Gold_Return_Category                           
Negative                  101      139      195
Positive                   50       77      117


In [25]:
chi2, test4_p_val, dof, expected = chi2_contingency(contingency_table)
print(f"Chi-Square Statistic: {chi2}, p-value: {test4_p_val}")

Chi-Square Statistic: 0.8621403265313069, p-value: 0.6498133161509081


In [26]:
test_hypothesis(test4_p_val)

'Fail to reject the null hypothesis.'

#### From p value of the test, we get to know that we fail to reject the null hypothesis.

#### We don't have sufficient evidences to justify that Gold bond returns are not independent of market conditions.